<a href="https://colab.research.google.com/github/Mohamed28112003/Muffakir/blob/main/Notebooks/HumanQAdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Extract text**

In [1]:
!pip install azure-ai-formrecognizer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.4/301.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.0 MB/s eta 0:00:00


In [2]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

# استبدل القيم الخاصة بك هنا
endpoint = "https://documentsfree.cognitiveservices.azure.com/"
api_key = "8ec69b60270f4900b487ab5eabf265ac"

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(api_key)
)


with open("/content/أسئلة اللجنة السباعية للنيابة العامة دفعة 2020.pdf", "rb") as document:
    poller = document_analysis_client.begin_analyze_document(
        model_id="prebuilt-layout",  # يمكنك استخدام نماذج مخصصة أو النماذج الجاهزة مثل layout أو invoices
        document=document
    )

result = poller.result()

print(f"Total pages: {len(result.pages)}")





Total pages: 31


In [3]:

with open("file_name.txt", "w", encoding="utf-8") as text_file:

    print(f"Total pages: {len(result.pages)}", file=text_file)

    for page in result.pages:
        print(f"\nPage number: {page.page_number}\n", file=text_file)

        for line in page.lines:
            print(line.content, file=text_file)

print("Text extracted and saved ")


Text extracted and saved 


**Convert Text to QA**

In [ ]:
import csv
import re

# Function to preprocess text into QA dataset
def preprocess_text(file_path):
    qa_data = []
    # unwanted_text = "إهداء لروح والدى الراجى عفو ربه / المستشار السيد أحمد عبدالله إسماعيل"
    question_pattern = re.compile(r"^س\s*\d+\)")  # Matches 'س' followed by a number and ')'

    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
        question, answer = "", ""

        for line in lines:
            line = line.strip()

            if line == unwanted_text or line.startswith("Page number"):
                continue

            if question_pattern.match(line):
                if question and answer:
                    qa_data.append((question, answer.strip()))
                question = line
                answer = ""
            else:
                answer += " " + line

        # Add the last Q&A pair
        if question and answer:
            qa_data.append((question, answer.strip()))

    return qa_data

text_file = "/content/file_name.txt"  # Update with your actual file text path
qa_data = preprocess_text(text_file)

csv_filename = "qa_dataset2.csv"

with open(csv_filename, mode="w", encoding="utf-8", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["question", "answer"])
    writer.writerows(qa_data)

print(f"CSV file '{csv_filename}' has been created successfully!")
